### Mach Probe ExB drift Analysis
#### <i>Reference<i> 
 - Rüdiger Back and Roger D. Bengtson, <i>A Langmuir/Mach probe array for edge plasma turbulence and flow<i>, 1997


In [20]:
from numpy import sin, pi, sqrt, arccos, log
from pandas import read_excel

e = 1.602e-19 # [C] electron charge
r_p = 0.15e-3 # [m] probe radius
l_p = 1.3e-3 # [m] probe length
h = 0.55e-3 # [m] Hole radius
s = 1.32e-3 # [m] Rotation center to Hole edge
R = 0.86e-3 # [m] Rotation center to Wire center
m_i = (40) * 1.67e-27 #[kg] mass of BF2+
k = 1.38e-23  #[m2kg/s2K] Boltzmann const
alpha = pi/2 # [rad] angle between B-field and Rotation center to Wire center
gamma = (1+0.5)/(2+0.5)
S_probe = pi*r_p*(r_p+2*pi*l_p)

In [21]:
class Machprobe():
    def __init__(self, ne, Te, Ti, m_i, I):
        self.ne = ne
        self.I = I
        self.Cs =sqrt(e*(Te+Ti)/(m_i))
        d_alpha = arccos((s**2 + R**2 - h**2)/(2*s*R))
        self.A_eff = l_p*(R*sin(alpha)+r_p-max(R*sin(alpha)-r_p, s*sin(alpha-d_alpha)))
        #print('Te : {} eV'.format(Te))
        #print('Ti : {} eV'.format(Ti))
        #print('Effective area : {} m2'.format(self.A_eff))
        #print('Ion sound speed : {} m/s'.format(self.Cs))
        
    def perp_current(self): 
        self.I_D = (r_p/l_p)*(1-gamma)*self.A_eff     # diffusion current calculation
        self.I_sat = gamma*e*self.A_eff*self.Cs*self.ne     # saturation current calculation
        self.I_perp = self.I - self.I_D - self.I_sat             # perpendicular current calculation
    
        #print('diffusion current : ',self.I_D)
        #print('saturation current : ',self.I_sat)
        #print('perp current : ',self.I_perp)

In [28]:
file_path = 'exp_data/'
name = '128G_Reverse.xlsx'
data = read_excel(file_path + name, encoding='cp1252')
data

,Unnamed: 0,pos,I_Upstream_electron [A],I_Downstream_electron [A],I_Upstream_ion [A],I_Downstream_ion [A],I_Upstream_electron_density [m-3],I_Downstream_electron_density [m-3],Electron temperature [eV],Ion temperature [eV],Estimated M_electron
0,-8,-8,27500,45500,145,283,376000000000000000,613000000000000000,7.64,0.1,457318.578027
1,-4,-4,50400,58400,475,558,686000000000000000,828000000000000000,7.46,0.1,134757.524859
2,0,0,62000,62100,506,506,864000000000000000,902000000000000000,7.10,0.1,1440.721088
3,4,4,60100,53700,413,364,844000000000000000,802000000000000000,7.10,0.1,-100551.697444
4,8,8,42900,35000,272,224,604000000000000000,566000000000000000,7.50,0.1,-186355.744870
5,12,12,20600,15200,118,113,340000000000000000,334000000000000000,5.82,0.1,-244171.373216


In [29]:
for i in range(len(data)):
    I_upstream_electron = data.loc[i,'I_Upstream_electron [A]']*S_probe
    I_downstream_electron = data.loc[i,'I_Downstream_electron [A]']*S_probe
    I_upstream_ion = data.loc[i,'I_Upstream_ion [A]']*S_probe
    I_downstream_ion = data.loc[i,'I_Downstream_ion [A]']*S_probe
    ne_upstream = data.loc[i,'I_Upstream_electron_density [m-3]']
    ne_downstream = data.loc[i,'I_Downstream_electron_density [m-3]']
    Te = data.loc[i,'Electron temperature [eV]']
    Ti = data.loc[i,'Ion temperature [eV]']
    
    
    upstream_electron = Machprobe(ne_upstream, Te, Ti, m_i, I_upstream_electron)
    downstream_electron = Machprobe(ne_downstream, Te, Ti, m_i, I_downstream_electron)
    upstream_ion = Machprobe(ne_upstream, Te, Ti, m_i, I_upstream_ion)
    downstream_ion = Machprobe(ne_downstream, Te, Ti, m_i, I_downstream_ion)

    upstream_electron.perp_current()
    downstream_electron.perp_current()
    upstream_ion.perp_current()
    downstream_ion.perp_current()
    
    data.loc[i,'Mach number_electron'] = 0.73*log(upstream_electron.I_perp/downstream_electron.I_perp)
    data.loc[i,'Mach number_ion'] = 0.73*log(upstream_ion.I_perp/downstream_ion.I_perp)
    #data.loc[i,'I_sat (Up) [A]'] = upstream.I_sat
    #data.loc[i,'I_D (Up) [A]'] = upstream.I_D
    #data.loc[i,'I_Perp (Up) [A]'] = upstream.I_perp
    #data.loc[i,'I_sat (Down) [A]'] = downstream.I_sat
    #data.loc[i,'I_D (Down) [A]'] = downstream.I_D
    #data.loc[i,'I_Perp (Down) [A]'] = downstream.I_perp
    
data.to_excel(file_path + 'No_ne_correction_Result_' + name,encoding='cp1252')